# comercio_ext_indices.tb_referencia_ncm_cgce
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/NCM_CGCE_delta`
## 📌 Descrição do arquivo

Classificação **CGCE** em estrutura hierárquica (N1 → N2 → N3), com descrições em **PT/EN/ES**.

|Coluna|Descrição técnica|Interpretação humana|
|---|---|---|
|`CO_CGCE_N3`|Código CGCE nível 3|Categoria econômica detalhada|
|`NO_CGCE_N3`|Nome CGCE N3 (PT)|Nome da categoria (PT)|
|`NO_CGCE_N3_ING`|Nome CGCE N3 (EN)|Nome da categoria (EN)|
|`NO_CGCE_N3_ESP`|Nome CGCE N3 (ES)|Nome da categoria (ES)|
|`CO_CGCE_N2`|Código CGCE nível 2|Categoria intermediária|
|`NO_CGCE_N2`|Nome CGCE N2 (PT)|Nome do grupo (PT)|
|`NO_CGCE_N2_ING`|Nome CGCE N2 (EN)|Nome do grupo (EN)|
|`NO_CGCE_N2_ESP`|Nome CGCE N2 (ES)|Nome do grupo (ES)|
|`CO_CGCE_N1`|Código CGCE nível 1|Macro-categoria|
|`NO_CGCE_N1`|Nome CGCE N1 (PT)|Macro-categoria (PT)|
|`NO_CGCE_N1_ING`|Nome CGCE N1 (EN)|Macro-categoria (EN)|
|`NO_CGCE_N1_ESP`|Nome CGCE N1 (ES)|Macro-categoria (ES)|

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from delta.tables import DeltaTable
jvm = spark._jvm
FileSystem = jvm.org.apache.hadoop.fs.FileSystem
Path = jvm.org.apache.hadoop.fs.Path
fs = FileSystem.get(spark._jsc.hadoopConfiguration())

In [0]:
bronzePath = "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/ncm_cgce/"
silverPath = "/mnt/silver/landingbeca2026jan/comercio_ext_indices/tb_referencia_ncm_cgce/"
silverTable = "silver_comercio_ext_indices.tb_referencia_ncm_cgce"

In [0]:
silverSchema = T.StructType([
    T.StructField("CO_CGCE_N3",   T.StringType(),   False),
    T.StructField("NO_CGCE_N3",   T.StringType(),   False),
    T.StructField("NO_CGCE_N3_ING", T.StringType(), True),
    T.StructField("NO_CGCE_N3_ESP", T.StringType(), True),
    T.StructField("CO_CGCE_N2",   T.StringType(),   True),
    T.StructField("NO_CGCE_N2",   T.StringType(),   True),
    T.StructField("NO_CGCE_N2_ING", T.StringType(), True),
    T.StructField("NO_CGCE_N2_ESP", T.StringType(), True),
    T.StructField("CO_CGCE_N1",   T.StringType(),   True),
    T.StructField("NO_CGCE_N1",   T.StringType(),   True),
    T.StructField("NO_CGCE_N1_ING", T.StringType(), True),
    T.StructField("NO_CGCE_N1_ESP", T.StringType(), True),
    T.StructField("TS_REF",       T.TimestampType(),False),
    T.StructField("NM_ORIGEM",    T.StringType(),   False),
])

## Extração
> #### **saprk.read**

In [0]:
df_bronze = spark.read.format("delta").load(bronzePath)

##Normalização
> #### **datatype**

In [0]:
def up(s): return F.upper(F.trim(F.col(s).cast(T.StringType())))

df_norm = (
    df_bronze
    .withColumn("CO_CGCE_N3", up("CO_CGCE_N3"))
    .withColumn("NO_CGCE_N3", F.col("NO_CGCE_N3").cast(T.StringType()))
    .withColumn("NO_CGCE_N3_ING", F.col("NO_CGCE_N3_ING").cast(T.StringType()))
    .withColumn("NO_CGCE_N3_ESP", F.col("NO_CGCE_N3_ESP").cast(T.StringType()))
    .withColumn("CO_CGCE_N2", up("CO_CGCE_N2"))
    .withColumn("NO_CGCE_N2", F.col("NO_CGCE_N2").cast(T.StringType()))
    .withColumn("NO_CGCE_N2_ING", F.col("NO_CGCE_N2_ING").cast(T.StringType()))
    .withColumn("NO_CGCE_N2_ESP", F.col("NO_CGCE_N2_ESP").cast(T.StringType()))
    .withColumn("CO_CGCE_N1", up("CO_CGCE_N1"))
    .withColumn("NO_CGCE_N1", F.col("NO_CGCE_N1").cast(T.StringType()))
    .withColumn("NO_CGCE_N1_ING", F.col("NO_CGCE_N1_ING").cast(T.StringType()))
    .withColumn("NO_CGCE_N1_ESP", F.col("NO_CGCE_N1_ESP").cast(T.StringType()))
    .withColumn("TS_REF", F.current_timestamp())
    .withColumn("NM_ORIGEM", F.lit("/landingbeca2026jan/comercio_ext_indices/NCM_CGCE_delta"))
)

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:
df_valid = df_norm.filter(F.col("CO_CGCE_N3").isNotNull())

In [0]:
df_dedup = df_valid.dropDuplicates(["CO_CGCE_N3"])

In [0]:
df_silver = df_dedup.select(
    "CO_CGCE_N3","NO_CGCE_N3","NO_CGCE_N3_ING","NO_CGCE_N3_ESP",
    "CO_CGCE_N2","NO_CGCE_N2","NO_CGCE_N2_ING","NO_CGCE_N2_ESP",
    "CO_CGCE_N1","NO_CGCE_N1","NO_CGCE_N1_ING","NO_CGCE_N1_ESP",
    "TS_REF","NM_ORIGEM"
)

##Carga
> #### **overwrite**

In [0]:
delta_target = DeltaTable.forName(spark, "silver_comercio_ext_indices.tb_referencia_ncm_cgce")

merge_condition = """
  t.CO_CGCE_N3 = s.CO_CGCE_N3 AND
  t.CO_CGCE_N2 = s.CO_CGCE_N2 AND
  t.CO_CGCE_N1 = s.CO_CGCE_N1
"""

(delta_target.alias("t")
    .merge(df_silver.alias("s"), merge_condition)
    .whenMatchedUpdate(set={
        "t.NO_CGCE_N3": "s.NO_CGCE_N3",
        "t.NO_CGCE_N3_ING": "s.NO_CGCE_N3_ING",
        "t.NO_CGCE_N3_ESP": "s.NO_CGCE_N3_ESP",
        "t.NO_CGCE_N2": "s.NO_CGCE_N2",
        "t.NO_CGCE_N2_ING": "s.NO_CGCE_N2_ING",
        "t.NO_CGCE_N2_ESP": "s.NO_CGCE_N2_ESP",
        "t.NO_CGCE_N1": "s.NO_CGCE_N1",
        "t.NO_CGCE_N1_ING": "s.NO_CGCE_N1_ING",
        "t.NO_CGCE_N1_ESP": "s.NO_CGCE_N1_ESP",
        "t.TS_REF": "s.TS_REF",
        "t.NM_ORIGEM": "s.NM_ORIGEM"
    })
    .whenNotMatchedInsertAll()
    .execute()
)